In [1]:
# import packages 
import xarray as xr 
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import scipy.io as sio
import datetime as datetime
import os
import cmocean as cm
%matplotlib inline
from scipy import stats
import gsw

In [13]:
ds = xr.open_dataset('/Users/mackenzie/Documents/Research/PhD/TNB_Research/data/P2_P3_2018/113142.nc')

In [19]:
def convert_time(start_time,var):
    """
    """
    base_time = np.datetime64(start_time)

    nanoseconds_per_day = 86_400_000_000_000

    time = base_time + (ds[var].values * nanoseconds_per_day).astype("timedelta64[ns]")

    return time 


In [20]:
convert_time(start_time="1950-01-01T00:00:00",var='Instrument_01_Time')

array([['2018-02-19T05:15:00.000000000', '2018-02-19T05:29:59.999996672',
        '2018-02-19T05:45:00.000003328', ...,
        '2019-01-17T15:14:59.999996672', '2019-01-17T15:30:00.000003328',
        '2019-01-17T15:45:00.000000000']], dtype='datetime64[ns]')

In [18]:
import numpy as np

# Assume ds['time'] is in units of "days since 1950-01-01 00:00:00"
base_time = np.datetime64("1950-01-01T00:00:00")

# Convert to timedelta in nanoseconds
# 1 day = 86400 seconds = 86_400_000_000_000 nanoseconds
nanoseconds_per_day = 86_400_000_000_000

# If ds['time'] is a DataArray of floats (days)
base_time + (ds['Instrument_01_Time'].values * nanoseconds_per_day).astype("timedelta64[ns]")


array([['2018-02-19T05:15:00.000000000', '2018-02-19T05:29:59.999996672',
        '2018-02-19T05:45:00.000003328', ...,
        '2019-01-17T15:14:59.999996672', '2019-01-17T15:30:00.000003328',
        '2019-01-17T15:45:00.000000000']], dtype='datetime64[ns]')

In [17]:
# Assume 'time' is in days since 1950-01-01
base_time = np.datetime64("1950-01-01T00:00:00")

# Convert to datetime64[ns]
base_time + ds['Instrument_01_Time']

UFuncTypeError: ufunc 'add' cannot use operands with types dtype('<M8[s]') and dtype('float64')

In [11]:
ds.Instrument_01_Time.values

array([[24886.21875   , 24886.22916667, 24886.23958333, ...,
        25218.63541667, 25218.64583333, 25218.65625   ]])

In [12]:
ds.Instrument_01_Time

<xarray.DataArray 'Instrument_01_Time' (SCALAR: 1, AQD_c1802RPB1_14296: 31915)> Size: 255kB
array([[24886.21875 , 24886.229167, 24886.239583, ..., 25218.635417,
        25218.645833, 25218.65625 ]])
Dimensions without coordinates: SCALAR, AQD_c1802RPB1_14296
Attributes:
    long_name:             Time
    unit:                  days since 1950-01-01 00:00:00
    Instrument file name:  c1802RPB1_14296

In [5]:
ds1 = xr.open_dataset('/Users/mackenzie/Documents/Research/PhD/TNB_Research/data/P2_P3_2018/113143.nc')

In [6]:
ds1

<xarray.Dataset> Size: 19MB
Dimensions:               (SCALAR: 1, AQD_c1802RPC1_14295: 31831,
                           c1802RPB1_14296: 31831,
                           SBE37_s1802RPC1_16417: 95487,
                           SBE37_s1802RPC2_16418: 95487,
                           SBE37_s1802RPC3_16419: 95487,
                           SBE56_t1802RPC1_2086: 222833,
                           Seagurad_c1802RPB1_1110: 6)
Dimensions without coordinates: SCALAR, AQD_c1802RPC1_14295, c1802RPB1_14296,
                                SBE37_s1802RPC1_16417, SBE37_s1802RPC2_16418,
                                SBE37_s1802RPC3_16419, SBE56_t1802RPC1_2086,
                                Seagurad_c1802RPB1_1110
Data variables: (12/37)
    lat                   (SCALAR) float64 8B ...
    lon                   (SCALAR) float64 8B ...
    Instrument_01_Time    (SCALAR, AQD_c1802RPC1_14295) float64 255kB ...
    Instrument_01_Press   (SCALAR, AQD_c1802RPC1_14295) float64 255kB ...
    Instrument_01_Temp    (SCALAR, AQD_c1802RPC1_14295) float64 255kB ...
    Instrument_01_Speed   (SCALAR, AQD_c1802RPC1_14295) float64 255kB ...
    ...                    ...
    Instrument_05_Time    (SCALAR, SBE56_t1802RPC1_2086) float64 2MB ...
    Instrument_05_Temp    (SCALAR, SBE56_t1802RPC1_2086) float64 2MB ...
    Instrument_06_Time    (SCALAR, Seagurad_c1802RPB1_1110) float64 48B ...
    Instrument_06_Tilt    (SCALAR, Seagurad_c1802RPB1_1110) float64 48B ...
    Instrument_06_Speed   (SCALAR, Seagurad_c1802RPB1_1110) float64 48B ...
    Instrument_06_dir     (SCALAR, Seagurad_c1802RPB1_1110) float64 48B ...
Attributes:
    Title:          P3 Mooring 2018-2019, Ross Sea Outflow Experiment
    Contact:        Melissa Bowen, m.bowen@auckland.ac.nz
    Creation Time:  27-Aug-2024
    Description:    Location: Ross Sea\nProject Leader: Melissa Bowen\nLatitu...
    Instrument_01:  Instrument Type: Nortek Aquadopp Current Meter (AQD)\nIns...
    Instrument_02:  Instrument Type: Seabird SBE-37\nInstrument Model: mcat\n...
    Instrument_03:  Instrument Type: Seabird SBE-37\nInstrument Model: mcat\n...
    Instrument_04:  Instrument Type: Seabird SBE-37\nInstrument Model: mcat\n...
    Instrument_05:  Instrument Type: Seabird SBE-56\nInstrument Model: sbe56\...
    Instrument_06:  Instrument Type: Aanderaa Seagurad\nInstrument Model: sea...